In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
import pandas as pd
df_csv = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
df_csv.columns = df_csv.columns.str.strip('$').str.strip()
print(df_csv.shape)
print(df_csv.columns.tolist())

(9243, 9)
['Code Evamed', 'Motif de demande', 'Code CIS', 'Code CIP', 'Dénomination spécialité', 'Date avis définitif', 'ASMR', 'Valeur ASMR', 'Libellé ASMR$']


In [3]:
# Check the data type of the column
print(df_csv['Valeur ASMR'].dtype)


object


In [4]:
import pandas as pd

# Load the Excel file
xls_file = 'Tableau_moleicules_et_marques_commercialisees_en_France_en_2022.xlsx'
xls = pd.ExcelFile(xls_file)

# Get the list of sheet names
sheet_names = xls.sheet_names
print("List of all sheet names in the Excel file:", sheet_names)


List of all sheet names in the Excel file: ['GERS 12 2022']


In [5]:
# Create a dictionary to hold each sheet's DataFrame
sheets_dict = pd.read_excel(xls_file, sheet_name=None)  # sheet_name=None loads all sheets into a dictionary

# Print the keys of the dictionary to confirm DataFrame names (each sheet is a DataFrame)
print("DataFrames created for each sheet:", sheets_dict.keys())


DataFrames created for each sheet: dict_keys(['GERS 12 2022'])


In [6]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///asmr.db")
print("Engine created for database 'asmr.db'")

Engine created for database 'asmr.db'


In [7]:
# Loop through each sheet in sheets_dict
for sheet_name, df in sheets_dict.items():
    # Clean up the sheet name to use it as a valid table name
    table_name = sheet_name.replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '').lower()
    
    # Load the DataFrame into the database as a separate table
    try:
        df.to_sql(table_name, engine, index=False, if_exists='replace')
        print(f"Successfully added '{sheet_name}' as table '{table_name}' in the database.")
    except Exception as e:
        print(f"Error while adding '{sheet_name}' as table '{table_name}': {e}")


Successfully added 'GERS 12 2022' as table 'gers_12_2022' in the database.


In [8]:
df_csv.to_sql("csv_data", engine, index=False, if_exists='replace')

9243

In [9]:
from sqlalchemy import create_engine, inspect

# Create an SQLite engine
engine = create_engine('sqlite:///asmr.db')

# Use the inspector to get a list of all tables in the database
inspector = inspect(engine)
tables = inspector.get_table_names()

# Print the list of tables
print("Existing tables:", tables)



Existing tables: ['PSP_SPE', 'age', 'asmr', 'atc1', 'atc2', 'atc3', 'atc4', 'atc5', 'ben_reg', 'cip13', 'csv_data', 'gen_num', 'gers_12_2022', 'sexe', 'top_gen', 'variables', 'xls_data']


In [10]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM asmr LIMIT 5;"))

sqlite
['PSP_SPE', 'age', 'asmr', 'atc1', 'atc2', 'atc3', 'atc4', 'atc5', 'ben_reg', 'cip13', 'csv_data', 'gen_num', 'gers_12_2022', 'sexe', 'top_gen', 'variables', 'xls_data']
[('$CT-21025$', '$Inscription (CT)$', '$66077218$', '$3400930294611$', '$FEXOFENADINE ARROW 180 mg, comprimé pelliculé$', '$04/09/2024$', '$ASMR$', '$V$', "$Ces spécialités sont des génériques qui n'apportent pas d'amélioration du service médical rendu (ASMR V) par rapport aux spécialités à base de fexofénadine déjà inscrites.$"), ('$CT-21016$', '$Inscription (CT)$', '$64314227$', '$3400955101734$', '$OXYGENE MEDICINAL LINDE HEALTHCARE 200 bar, gaz pour inhalation, en bouteille$', '$28/08/2024$', '$ASMR$', '$V$', "$Cette spécialité est un complément de gamme qui n'apporte pas d'amélioration du service médical rendu (ASMR V) par rapport aux présentations déjà inscrites.$"), ('$CT-21007$', '$Inscription (CT)$', '$60120260$', '$3400930280522$', '$CHLORHEXIDINE LIBERTY PHARMA 0,20 %, solution pour bain de bouche$', 

In [11]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [12]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
agent_executor.invoke({"input: give me the code EAN of ZYTIGA 250MG CPR FL 120"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


PSP_SPE, age, asmr, atc1, atc2, atc3, atc4, atc5, ben_reg, cip13, csv_data, gen_num, gers_12_2022, sexe, top_gen, variables, xls_data
Invoking: `sql_db_schema` with `{'table_names': 'PSP_SPE'}`



CREATE TABLE "PSP_SPE" (
	"PSP_SPE" FLOAT, 
	"Libellé Prescripteur" TEXT, 
	"Unnamed: 2" TEXT
)

/*
3 rows from PSP_SPE table:
PSP_SPE	Libellé Prescripteur	Unnamed: 2
None	None	None
1.0	MEDECINE GENERALE LIBERALE	PRESCRIPTEURS DE VILLE
(MEDECINS et DENTISTES)
2.0	ANESTHESIOLOGIE - REANIMATION LIBERALE	None
*/
Invoking: `sql_db_schema` with `{'table_names': 'cip13'}`



CREATE TABLE cip13 (
	"CIP13" BIGINT, 
	"Libellé CIP13" TEXT
)

/*
3 rows from cip13 table:
CIP13	Libellé CIP13
3400921604696	YELLOX 0,9MG/ML COLLY FL5ML 1
3400921609080	AVONEX 0,03MG/0,5ML SOL INJ STYLO 4
3400921610550	POLYGYNAX VIRGO CAPSULE VAGINALE 6
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT CIP13 FROM cip13 WHERE Libe

{'input': {'input: give me the code EAN of ZYTIGA 250MG CPR FL 120'},
 'output': 'It appears that there were no results found for "ZYTIGA 250MG CPR FL 120" in the database. Therefore, I cannot provide the EAN code for that product.'}